In [1]:
from bs4 import BeautifulSoup
import lxml
import re
import os.path
my_path = os.path.abspath('')

In [2]:
#Data cleaning approach:

#First identify the table of content by regex to get mention of "Item" in <Table> <tbody> tag(perhaps extract the table of content)
#or search <a href="#....." util the mention of Item16, signautre
#extract all the href in <a> attribute in the table of content (ex. href1, href2, href3 …)

#Second have algorithm to identify <a name="href1"> and save the content between href1 and href2, href2 and href3, …

#Third, clean the html tags in the contents 

In [3]:
def load_html(path):
    return BeautifulSoup(open(path,"r"), 'html.parser')
def load_xml(path):
    return BeautifulSoup(open(path,"r"), 'lxml')

#get the text from soup
def soup_get_text(soup):
    print(soup.get_text())

#write "string" to file from "path"
def write_file(path, string):
    text_file = open(path, "w")
    text_file.write(string)
    text_file.close()


#print elements from list
def print_list(lists):
    for list in lists:
        print(list)

#read all the text file in the directory
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))
                
    return files


In [4]:
#First extract the location of the table of content by
#using regex to identify "Item 15" or "Financial Statement Schedules"
#get every links <a href="#"> above "Item 15" or "Financial Statement Schedules"
#only works with the text and items in table of content contains href#
#if only in the page number, doesn't work
def get_table_of_content_links(soup):
    tag = soup.find('a',string=re.compile("Financial Statement Schedule"))
    tag_cap = soup.find('a',string=re.compile("FINANCIAL STATEMENT SCHEDULE"))
    tag_item = soup.find('a',string=re.compile("Item 15"))
    if tag==None and tag_item!=None:
        tag = tag_item
    if tag==None and tag_cap!=None:
        tag = tag_cap
    
    links = list()
    links.append(tag.get('href')[1:])
    for elements in tag.previous_elements:
        if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
            #print(elements)
            links.append(elements.get('href')[1:])
    
    links.reverse()
    print("total section: ",len(remove_duplicates_in_list(links)))
    return remove_duplicates_in_list(links)

#remove dupicated items
def remove_duplicates_in_list(List):
    return list(dict.fromkeys(List))
"""
def get_table_of_content_links2(soup):
    links = list()
    for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
        if link.get('href')!=None and link.get('href')[0]=='#':
            links.append(link.get('href')[1:])
    return links
# or identify the signature link and use find_previous method to extract:extract()?
"""


"\ndef get_table_of_content_links2(soup):\n    links = list()\n    for link in soup.find_all('a'):\n    #link.get('href')\n    #print(link.get('href'))\n    #print(link.get('name'))\n        if link.get('href')!=None and link.get('href')[0]=='#':\n            links.append(link.get('href')[1:])\n    return links\n# or identify the signature link and use find_previous method to extract:extract()?\n"

In [5]:
#get every sections by using the links to identify <a>
def get_all_sections(links, soup):
    sections=list()
    for i in range(len(links)):
        if i<len(links)-1:
            sections.append(get_section(links[i],links[i+1],soup))
        else:
            sections.append(get_last_section(links[i],soup))
    return sections

#get sections between links[i] and links[i+1]
def get_section(link1, link2, soup):
    section_str=""
    #need to think of <p id
    #need to think of <div id=""
    #need to think of <span id=""
    #need to think of <td id=""
    
    for link in soup.find_all('p'):
        if link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='p' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    for link in soup.find_all('div'):
        if link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='div' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    for link in soup.find_all('span'):
        if link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='span' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    for link in soup.find_all('td'):
        if link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='td' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('name') and elements['name']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

#get last section that is between <a name="links[n]"> to the end of the document
def get_last_section(link1,soup):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

In [6]:
#Remove all the tag that is not <type>10-k using xml
def clean_10k_by_xml(path):
    soup=load_xml(path)
    extract_type_10k(soup)
    write_file(path, str(soup))

#remove all tag that is not <type>10-k
def extract_type_10k(soup):
    tag_list=soup.find_all('type')
    for tag in tag_list:
        print(tag.next_element)
        if '10-K' not in tag.next_element:
            tag.extract()
    

In [7]:
#First remove all the tag that is not <type>10-k including xml
#Secont extract the links of the items from the table of content 
#Third get all the sections and store it in a list
def get_10k_sections(path):
    soup=load_html(path)
    sections=get_all_sections(get_table_of_content_links(load_html(path)),soup)
    return sections

#get all the path of 10k file and store in list name files
def get_10k_files():
    files = [];
    files = file_reader(my_path+"/sec_edgar_filings")
    return files

In [8]:
#print(sections[0])
def save_sections(tenk_filings):
    report_number = 0
    section_num =0
    for report in tenk_filings:
        if not os.path.exists(my_path+'/clean_data/filing_'+str(report_number)):
            os.mkdir(my_path+'/clean_data/filing_'+str(report_number))
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " Created ")
        else:    
            print("Directory " , '/clean_data/filing_'+str(report_number) ,  " already exists")
        
        for section in report:
            write_file('clean_data/filing_'+str(report_number)+'/filing_'+str(report_number)+'_section_'+str(section_num)+".txt",section)
            section_num = section_num+1
        report_number = report_number+1

def save_sections(filing, path, report_number):
    section_num =0
    for section in filing:
        write_file(path+'/'+report_number+'_section_'+str(section_num)+".txt",section)
        section_num = section_num+1


def save_filings_by_section(ticker, filings, path):
    
    if not os.path.exists(path+'/clean_data'):
        os.mkdir(path+'/clean_data')
        print("Directory " , '/clean_data/',  " Created ")
    for filing in range(len(tenk_filings)):
        if not os.path.exists(path+'/clean_data/'+ticker[filing]):
            os.mkdir(path+'/clean_data/'+ticker[filing])
            print("Directory " , '/clean_data/'+ticker[filing] ,  " Created ")
        else:    
            print("Directory " , '/clean_data/'+ticker[filing] ,  " already exists")
        
        
        save_sections(tenk_filings[filing],path+'/clean_data/'+ticker[filing], ticker[filing])
        



In [9]:
import time
start_time = time.time()

In [10]:
files_path=list()
files_path=get_10k_files()
files_path.sort()
print_list(files_path)
company_list = list()


/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/ACIW/10-K/0000935036-20-000009.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/ADS/10-K/0001101215-20-000049.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AKAM/10-K/0001086222-20-000045.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/ALRM/10-K/0001459200-20-000005.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/AMZN/10-K/0001018724-20-000004.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/BCOR/10-K/0001068875-20-000056.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/BKI/10-K/0001627014-20-000005.txt
/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data

In [11]:

import pandas as pd


df = pd.read_csv("DGTL_holdings.csv") 
company_list = list()
for index, row in df.iterrows():
    if row['Location'] == "United States" and row["Exchange"]!="-" and row["Issuer Ticker"]!="PAGS" and row["Issuer Ticker"]!="SPOT" and row["Issuer Ticker"]!="GOOGL" and row["Issuer Ticker"]!="CVNA" and row["Issuer Ticker"]!="EBAY" and row["Issuer Ticker"]!="ETSY" and row["Issuer Ticker"]!="GRPN" and row["Issuer Ticker"]!="JKHY" and row["Issuer Ticker"]!="UPWK" and row["Issuer Ticker"]!="WEX":
        company_list.append(row['Issuer Ticker'])
company_list.sort()
print_list(company_list)
#googl is class A, PAGS and SPOT dont have 10k, 

ACIW
ADS
AKAM
ALRM
AMZN
BCOR
BKI
CARG
CARS
CI
CONE
CTXS
DBX
DOCU
DT
EBIX
EQIX
EVBG
EVOP
EVTC
FB
FDX
FEYE
FFIV
FICO
FIS
FISV
FLT
FLWS
FTNT
GDDY
GDOT
GOOG
GPN
GWRE
IAA
IAC
IDCC
JCOM
JNPR
MA
MELI
MIME
MTCH
NEWR
NFLX
NLOK
NTAP
NTCT
OSTK
PANW
PBI
PFPT
PINS
PSN
PYPL
QLYS
QNST
QTS
QTWO
QUOT
REAL
ROP
SFIX
SQ
SSNC
SWCH
SWI
TENB
TRIP
TWTR
UPS
V
VMW
W
WSM
YELP
Z
ZG
ZS


In [12]:
for file in files_path:
    print(file)
    clean_10k_by_xml(file)


/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/ACIW/10-K/0000935036-20-000009.txt
10-K

EX-21.01

EX-23.01

EX-31.01

EX-31.02

EX-32.01

EX-32.02

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

GRAPHIC

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/ADS/10-K/0001101215-20-000049.txt
10-K

EX-4.2

EX-10.26

EX-10.27

EX-10.125

EX-21

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

GRAPHIC

GRAPHIC

EX-101.SCH

EX-101.

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/CONE/10-K/0001553023-20-000025.txt
10-K

EX-4.5

EX-10.36

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

GRAPHIC

GRAPHIC

XML

XML

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/CTXS/10-K/0000877

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/EVBG/10-K/0001564590-20-007797.txt
10-K

EX-4.5

EX-21.1

EX-23.1

EX-23.2

EX-31.1

EX-31.2

EX-32.1

EX-32.2

GRAPHIC

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/EVOP/10-K/0001558370-20-001658.txt
10-K

EX-4.2

EX-10.32

EX-10.33

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/FLT/10-K/0001175454-20-000008.txt
10-K

EX-4.2

EX-10.53

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

GRAPHIC

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/FLWS/10-K/0001437749-19-018360.txt
10-K

EX-4.2

EX-21.1

EX-23.1

EX-31.1

E

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/IDCC/10-K/0001405495-20-000007.txt
10-K

EX-4.2

EX-10.20

EX-10.31

EX-21

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

GRAPHIC

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/JCOM/10-K/0001084048-20-000012.txt
10-K

EX-4.5

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-1

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/NTAP/10-K/0001564590-20-029349.txt
10-K

EX-4.5

EX-10.14

EX-10.19

EX-10.23

EX-10.24

EX-10.56

EX-10.57

EX-10.58

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

GRAPHIC

GRAPHIC

GRAPHIC

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

/Users/davidren

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/QNST/10-K/0001564590-19-033605.txt
10-K

EX-23.1

EX-31.1

EX-31.2

EX-32.1

GRAPHIC

EX-101.INS

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

EXCEL

XML

XML

XML

ZIP

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/QTS/10-K/0001558370-20-001789.txt
10-K

EX-3.7

EX-4.12

EX-10.33

EX-10.34

EX-10.35

EX-10.36

EX-10.45

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-31.3

EX-31.4

EX-32.1

EX-32.2

GRAPHIC

GRAPHIC

EX-101.SCH

EX

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/TENB/10-K/0001660280-20-000012.txt
10-K

EX-4.3

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

XML

XML

XML

XML

EXCEL

XML

XML

XML

XML

JSON

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/TRIP/10-K/0001564590-20-005269.txt
10-K

EX-4.2

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-32.2

GRAPHIC

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

XM

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/YELP/10-K/0001345016-20-000009.txt
10-K

EX-10.9

EX-10.12

EX-10.17

EX-21.1

EX-23.1

EX-31.1

EX-31.2

EX-32.1

EX-101.SCH

EX-101.CAL

EX-101.DEF

EX-101.LAB

EX-101.PRE

GRAPHIC

GRAPHIC

XML

EXCEL

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

JSON

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

ZIP

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

XML

/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/Z/10-K/0001617640-20-000015.txt
10-K

EX-4.17

EX-21.1

EX-2

In [13]:
sections=list()
tenk_filings = list()
for file in files_path:
    if file!=files_path[12] and file!=files_path[16] and file!=files_path[19] and file!=files_path[37] and file!=files_path[43] and file!=files_path[77] and file!=files_path[81]:
        sections=get_10k_sections(file)
        tenk_filings.append(sections)

#save_filings_by_section(tenk_filings, my_path)
#CVNA, EBAY, ETSY, GRPN, JKHY, UPWK, WEX has ref in page number
#CARG, CARS, MIME,NTAP, NTCT, PFPT, QUOT(in table), SSNC, TRIP(in table) < p id=""
#CI, CTXS, DOCU, GDDY, MTCH, PINS,NEWR, SQ,TWTR, YELP, Z, ZG, ZS <div id=""
#MELI <span id=""
#WSM <td id=""



total section:  25
total section:  26
total section:  21
total section:  27
total section:  21
total section:  24
total section:  20
total section:  24
total section:  20
total section:  65
total section:  16
total section:  16
total section:  21
total section:  25
total section:  24
total section:  25
total section:  24
total section:  24
total section:  26
total section:  25
total section:  26
total section:  21
total section:  24
total section:  24
total section:  25
total section:  35
total section:  22
total section:  22
total section:  25
total section:  21
total section:  26
total section:  21
total section:  22
total section:  17
total section:  26
total section:  25
total section:  44
total section:  25
total section:  24
total section:  23
total section:  21
total section:  26
total section:  21
total section:  44
total section:  21
total section:  21
total section:  21
total section:  20
total section:  21
total section:  21
total section:  21
total section:  20
total sectio

In [14]:
save_filings_by_section(company_list, tenk_filings, my_path)

Directory  /clean_data/ACIW  already exists
Directory  /clean_data/ADS  already exists
Directory  /clean_data/AKAM  already exists
Directory  /clean_data/ALRM  already exists
Directory  /clean_data/AMZN  already exists
Directory  /clean_data/BCOR  already exists
Directory  /clean_data/BKI  already exists
Directory  /clean_data/CARG  already exists
Directory  /clean_data/CARS  already exists
Directory  /clean_data/CI  already exists
Directory  /clean_data/CONE  already exists
Directory  /clean_data/CTXS  already exists
Directory  /clean_data/DBX  already exists
Directory  /clean_data/DOCU  already exists
Directory  /clean_data/DT  already exists
Directory  /clean_data/EBIX  already exists
Directory  /clean_data/EQIX  already exists
Directory  /clean_data/EVBG  already exists
Directory  /clean_data/EVOP  already exists
Directory  /clean_data/EVTC  already exists
Directory  /clean_data/FB  already exists
Directory  /clean_data/FDX  already exists
Directory  /clean_data/FEYE  already exist

In [15]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 2374.3843669891357 seconds ---


In [ ]:
#BLK file 20 link in <p id=" "
#visa file 2015 have link from item 1-14 but probably forgot to add link to < a item 15 or < a Financial Statement Schedules
#visa file 2015 add link to the page

In [ ]:
#remove all xml tag and save it 
"""
remove_xml(soup1)
text_file = open("sec_edgar_filings/clean_data/sample_10k.txt", "w")
text_file.write(str(soup1))
text_file.close()
"""
#load html soup and prettify
"""
soup=load_html("sec_edgar_filings/clean_data/sample_10k.txt")
soup.prettify()
"""
#get table of content links
"""
links = get_table_of_content_links1(soup)
print_list(links)
print(len(links))
"""
#get all sections in 10k
"""
sections=list()
get_all_sections(sections,links)
print(sections[21])
"""
#write sections to a txt file
"""
text_file = open("sample_KG.txt", "w")
text_file.write(sections[2])
text_file.close()
"""

In [ ]:
"""
MSFT_1_path='sec_edgar_filings/MSFT/10-K/0001193125-15-272806.txt'
MSFT_2_path="sec_edgar_filings/MSFT/10-K/0001564590-18-019062.txt"
APPL_1_path='sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt'
APPL_2_path='sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt'
VISA_1_path='sec_edgar_filings/V/10-K/0001403161-18-000055.txt'
VISA_2_path='sec_edgar_filings/V/10-K/0001403161-19-000050.txt'
GOOG_1_path='sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt'
GOOG_2_path='sec_edgar_filings/GOOG/10-K/0001652044-20-000008.txt'
BLK_1_path='sec_edgar_filings/BLK/10-K/0001564590-19-005479.txt'
BLK_2_path='sec_edgar_filings/BLK/10-K/0001564590-20-007807.txt'
BA_1_path='sec_edgar_filings/BA/10-K/0000012927-19-000010.txt'
BA_2_path='sec_edgar_filings/BA/10-K/0000012927-20-000014.txt'
FORD_1_path='sec_edgar_filings/F/10-K/0000037996-20-000010.txt'
FORD_2_path='sec_edgar_filings/F/10-K/0000037996-19-000012.txt'

soup=load_html(MSFT_1_path)
"""
#soup_get_text(soup)
#visa, msft, goog, blk work
#, ba_1_path uses <a id="links">
#ford dont work cuz href is at the page number in toc


In [ ]:
"""
#get table of content

"identify the mention of Signatures"
tag = soup.find('a',string=re.compile("Signatures"))
#print(tag)
links = list()
for elements in tag.previous_elements:
    if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
        #print(elements)
        links.append(elements.get('href')[1:])
        
links.reverse()
#print_list(links)
"""

In [ ]:
#load the html
"""
soup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), 'html.parser')
#print(soup.prettify())
print(soup.get_text())
"""



In [ ]:
"""
#get all href from the table of content 
Links = list()
for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
    if link.get('href')!=None and link.get('href')[0]=='#':
        Links.append(link.get('href')[1:])
for link in Links:
    print(link)
        
    #remove the '#' and store them in a list
"""

In [ ]:
#get section 
#example get section between Item 1 and Item 1A

#Problem also get the sentences
"""
Sections=""
for link in soup.find_all('a'):
    if link.has_attr('name') and link['name']=='ITEM_1_BUSINESS':
        for elements in link.next_elements:
            if elements.name=='a' and elements.has_attr('name') and elements['name']=='ITEM_1B_UNRESOLVED_STAFF_COMMENTS':
                break
                
            
            if str(elements)[0] != '<':
                Sections = Sections + str(elements)
                print(repr(elements))
            #Sections = Sections + str(elements)
            
        break
"""


for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('name') and elements['name']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + "\n" +str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break